##### Author: Bishesh Kafle
##### Date : 2024-07-22

In [1]:
from kafka import KafkaProducer
import time
import numpy as np
from utils.mysql_connection import *

In [2]:
# Load stream_data table form kafka_prod database
df = table_df('kafka_prod','stream_data')
df

/Users/bses/F1_Intern/Kafka_Streaming/utils/mysql_connection.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=con.connection)


,tran_date,account_number,branch,product,lcy_amount,transaction_code,description1,dc_indicator,is_salary
0,2020-06-09,02XYZXYZ10017529992,15,SBA,14500,CI,ATM WDL /00463701/NABILNABIL,withdraw,0\r
1,2020-06-14,02XYZXYZ10017529992,15,SBA,20000,CI,ATM WDL /00463701/NABILNABIL,withdraw,0\r
2,2020-06-01,02XYZXYZ10017517823,15,SBA,4000,CI,ATM WDL /NICAV701/PharpingKT,withdraw,0\r
3,2020-06-05,02XYZXYZ10017517823,15,SBA,500,CI,ATM WDL /NICAV701/PharpingKT,withdraw,0\r
4,2020-06-12,02XYZXYZ10017517823,15,SBA,5000,CI,ATM WDL /NICAV701/PharpingKT,withdraw,0\r
...,...,...,...,...,...,...,...,...,...
45026,2020-08-16,02XYZXYZ10017509810,17,SBA,1000,BI,CASBA allot of LAXMI UNNATI KOSH,withdraw,0\r
45027,2020-08-16,02XYZXYZ100172122201,17,SBA,1000,BI,CASBA allot of LAXMI UNNATI KOSH,withdraw,0\r
45028,2020-08-16,02XYZXYZ10017520310,17,SBA,5000,BI,CASBA allot of LAXMI UNNATI KOSH,withdraw,0\r
45029,2020-08-16,02XYZXYZ10017559076,17,SBA,0.38,IP,^0210017559076^:Int.Pd:^16-07-2020^ to ^16-08-202,deposit,0\r


In [3]:
# Assign Kafka topic name and bootstrap server address
KAFKA_TOPIC_NAME_CONS = "Topic1"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

# Start Producer Application
if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    # Initialize KafkaProducer object with specified bootstrap server and value serializer
    kafka_producer_obj = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
        value_serializer=lambda x: x.encode('utf-8')
    )
    
    # Add an order_id column with unique identifiers for each row
    df['order_id'] = np.arange(len(df))

    # Convert the DataFrame to a list of dictionaries, each representing a record
    data = df.to_dict(orient="records")
       
    # Iterate through each record in the data
    for message in data:
        # Create a list and store field values of the message
        message_fields_value_list = [message["order_id"], message["account_number"], message["branch"], message["transaction_code"]]

        # Concatenate field values into a single string separated by commas
        message = ','.join(str(v) for v in message_fields_value_list)

        # Print the type and content of the message for debugging purposes
        print("Message Type: ", type(message))
        print("Message: ", message)

        # Send the message to the specified Kafka topic
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)

        # Pause for a short time before sending the next message
        time.sleep(1)

    print("Kafka Producer Application Completed. ")
    

Kafka Producer Application Started ... 
Message Type:  <class 'str'>
Message:  0,02XYZXYZ10017529992,15,CI
Message Type:  <class 'str'>
Message:  1,02XYZXYZ10017529992,15,CI
Message Type:  <class 'str'>
Message:  2,02XYZXYZ10017517823,15,CI
Message Type:  <class 'str'>
Message:  3,02XYZXYZ10017517823,15,CI
Message Type:  <class 'str'>
Message:  4,02XYZXYZ10017517823,15,CI
Message Type:  <class 'str'>
Message:  5,02XYZXYZ10017519116,15,CI
Message Type:  <class 'str'>
Message:  6,02XYZXYZ10017519121,15,CI
Message Type:  <class 'str'>
Message:  7,02XYZXYZ10017520752,15,CI
Message Type:  <class 'str'>
Message:  8,02XYZXYZ10017520752,15,CI
Message Type:  <class 'str'>
Message:  9,02XYZXYZ10017520752,15,CI
Message Type:  <class 'str'>
Message:  10,02XYZXYZ10017555515,15,CI
Message Type:  <class 'str'>
Message:  11,02XYZXYZ10017555612,15,CI
Message Type:  <class 'str'>
Message:  12,02XYZXYZ10017557070,15,O
Message Type:  <class 'str'>
Message:  13,02XYZXYZ10017557090,15,CI
Message Type:  <cla

KeyboardInterrupt: 